In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from functools import partial

In [ ]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

In [ ]:
def generalRegressionPredictor(self, transformedData):
    return self.predict(transformedData)

In [ ]:
spamDf = pd.read_excel("./Spam.xlsx")
# spamDf = pd.read_excel("https://www.dropbox.com/scl/fi/v24mmhg5hmefmnv99uqsy/Spam.xlsx?rlkey=iq7exnueq84sy7y2b8ud70mp0&dl=1")
spamDf

In [ ]:
spamDf.size, spamDf.shape

In [ ]:
evaluationBinaryCalssifiactionSampleData = {
    "Recipients": [19, 15, 13],
    "Hyperlinks": [1, 1, 11],
    "Characters": [47, 58, 88]
}

In [ ]:
def evaluationBinaryCalssifiactionTransformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    if isinstance(dataForTransfer, pd.DataFrame):
        df = dataForTransfer.copy()
    else:
        df = pd.DataFrame(dataForTransfer)
    dfTransformer = sm.add_constant(df[["Recipients", "Hyperlinks", "Characters"]],has_constant='add')
    return dfTransformer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the dataset into training and testing sets
trainSet, testSet = train_test_split(
  spamDf,
  test_size=0.3,
  random_state=1,
  stratify=spamDf['Spam']
)
trainSet.shape, testSet.shape

In [ ]:
# Fit the logistic regression model
features = ['Recipients', 'Hyperlinks', 'Characters']
xTrain = trainSet[features]
yTrain = trainSet['Spam'].astype(int)

In [ ]:
model1 = sm.Logit(
  yTrain,
  evaluationBinaryCalssifiactionTransformer(xTrain)
)
model1Fit = model1.fit()
print(model1Fit.summary())

In [ ]:
predict1 = model1Fit.predict(evaluationBinaryCalssifiactionTransformer(testSet))
testSet['predict1'] = predict1
sumTable = pd.DataFrame({'A': testSet['Spam'], 'Prob': testSet['predict1']})
sumTable.to_csv("ROC.csv", index=True)


In [ ]:
sumTable1 = pd.DataFrame({'A': testSet['Spam'], 'Prob': testSet['predict1']})

In [ ]:
# Make predictions based on probability threshold of 0.5
testSet['predictions'] = (testSet['predict1'] > 0.5).astype(int)
sumTable1['P'] = testSet['predictions']
sumTable1

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(sumTable1['A'], sumTable1['P'])
print(f'Accuracy: {accuracy}')

In [ ]:
# Calculate recall
recall = recall_score(sumTable1['A'], sumTable1['P'])
print(f'Recall: {recall}')

In [ ]:
# Calculate precision
precision = precision_score(sumTable1['A'], sumTable1['P'])
print(f'Precision: {precision}')

In [ ]:
# Sensitivity and Specificity (Sensitivity is same as recall)
sensitivity = recall
specificity = sum((sumTable1['A'] == 0) & (sumTable1['P'] == 0)) / sum(sumTable1['A'] == 0)
print(f'Sensitivity: {sensitivity}')
print(f'Specificity: {specificity}')

In [ ]:
# Calculate F1 Score
f1Score = 2 * (precision * recall) / (precision + recall)
print(f'F1 Score: {f1Score}')

In [ ]:
# Plot ROC curve
fpr, tpr, _ = roc_curve(testSet['Spam'], testSet['predict1'])
roc_auc = roc_auc_score(testSet['Spam'], testSet['predict1'])
# Calculate AUC
print(f'AUC: {roc_auc}')


In [ ]:
import matplotlib.pyplot as plt

In [ ]:

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
model1Fit.transformer = evaluationBinaryCalssifiactionTransformer
model1Fit.mainPredictor = partial(generalRegressionPredictor, model1Fit)
registerAJrjModel(
    model1Fit,
    {
        "modelName":f"evaluationBinaryCalssifiactionModelWithExteraMetircs",
        "version":"1.0.1",
        "params": model1Fit.params.to_dict(),
        "score": accuracy,
        "otherEvaluationMetrics": {
            "accuracy": accuracy,
            "recall": recall,
            "precision": precision,
            "sensitivity": sensitivity,
            "specificity": specificity,
            "f1Score": f1Score,
            "roc_auc": roc_auc,
        },
        "modelLibrary": "statsmodels.api.Logit",
        "libraryMetadata": {
            "pvalues": model1Fit.pvalues.to_dict(),
            "pseudo_r_squared": float(model1Fit.prsquared),
            "llf": float(model1Fit.llf),
            "aic": float(model1Fit.aic),
            "bic": float(model1Fit.bic)
        },
    
        "sampleData": {
            "dataForTransfer": evaluationBinaryCalssifiactionSampleData
        }
    }
)